# CADS API constraints tests

In [40]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [41]:
import os
import xarray as xr

import cads_api_client
import cads_processing_api_service

In [32]:
api_url = os.getenv("CADS_API_ROOT_URL", "http://cds2-dev.bopen.eu/api")
api_url

'http://cds2-dev.bopen.eu/api'

## Client instantiation

The client exposes the APIs for:
- Catalogue exploration (see **Section 1**) 
- Data retrieval (see **Section 2**)
- Constraints validation

In [33]:
client = cads_api_client.ApiClient(api_url, api_key="mysecretpat")
client

ApiClient(url='http://cds2-dev.bopen.eu/api', api_key='mysecretpat')

In [43]:
collection = client.collection("reanalysis-era5-pressure-levels")
collection.json

{'type': 'Collection',
 'id': 'reanalysis-era5-pressure-levels',
 'stac_version': '1.0.0',
 'title': 'ERA5 hourly data on pressure levels from 1959 to present',
 'description': '**ERA5** is the fifth generation ECMWF reanalysis for the global climate and weather for the past 4 to 7 decades.\nCurrently data is available from 1950, with Climate Data Store entries for 1950-1978 (preliminary back extension) and from 1959 onwards (final release plus timely updates, this page).\nERA5 replaces the ERA-Interim reanalysis.\n\nReanalysis combines model data with observations from across the world into a globally complete and consistent dataset using the laws of physics. This principle, called data assimilation, is based on the method used by numerical weather prediction centres, where every so many hours (12 hours at ECMWF) a previous forecast is combined with newly available observations in an optimal way to produce a new best estimate of the state of the atmosphere, called analysis, from which

## Constraints validation

In [36]:
cads_processing_api_service.constraints.validate_constraints()